In [1]:
import ngram
import pandas as pd
import numpy as np
import dill 
import pickle
from helpers import *
from TfidfFeatureGenerator import *
from CountFeatureGenerator import *
from SvdFeatureGenerator import *
from Word2VecFeatureGenerator import *
from SentimentFeatureGenerator import *

C:\Users\shrey\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\shrey\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
    read = True
    if not read:
        body_train = pd.read_csv("fnc-1/train_bodies_processed.csv", encoding='utf-8')
        stances_train = pd.read_csv("fnc-1/train_stances_processed.csv", encoding='utf-8')
        train = pd.merge(stances_train, body_train, how='left', on='Body ID')
        targets = ['agree', 'disagree', 'discuss', 'unrelated']
        targets_dict = dict(zip(targets, range(len(targets))))
        train['target'] = map(lambda x: targets_dict[x], train['Stance'])
        print('train.shape:')
        print(train.shape)
        n_train = train.shape[0]
        
        data = train
        # read test set, no 'Stance' column in test set -> target = NULL
        # concatenate training and test set
        test_flag = True
        if test_flag:
            body_test = pd.read_csv("fnc-1/test_bodies_processed.csv", encoding='utf-8')
            headline_test = pd.read_csv("fnc-1/test_stances.csv", encoding='utf-8')
            test = pd.merge(headline_test, body_test, how="left", on="Body ID")
            
            data = pd.concat((train, test)) # target = NaN for test set
            print(data)
            print('data.shape:')
            print(data.shape)
            
            train = data[~data['target'].isnull()]
            print(train)
            print('train.shape:')
            print(train.shape)
            
            test = data[data['target'].isnull()]
            print(test)
            print('test.shape:')
            print(test.shape)
            
            
        print("generate unigram")
        data["Headline_unigram"] = data["Headline"].map(lambda x: preprocess_data(x))
        data["articleBody_unigram"] = data["articleBody"].map(lambda x: preprocess_data(x))

        print("generate bigram")
        join_str = "_"
        data["Headline_bigram"] = data["Headline_unigram"].map(lambda x: ngram.getBigram(x, join_str))
        data["articleBody_bigram"] = data["articleBody_unigram"].map(lambda x: ngram.getBigram(x, join_str))
        
        print("generate trigram")
        join_str = "_"
        data["Headline_trigram"] = data["Headline_unigram"].map(lambda x: ngram.getTrigram(x, join_str))
        data["articleBody_trigram"] = data["articleBody_unigram"].map(lambda x: ngram.getTrigram(x, join_str))
            
        with open('data.pkl', 'wb') as outfile:
            dill.dump(data, outfile, -1)
            print('dataframe saved in data.pkl')
    
    else:
        with open('data.pkl', 'rb') as infile:
            data = dill.load(infile)
            print('data loaded')
            print('data.shape:')
            print(data.shape)
            
    


data loaded
data.shape:
(75385, 11)


In [18]:
    countFG = CountFeatureGenerator()
    tfidf = TfidfFeatureGenerator()
    svdFG = SvdFeatureGenerator()
    word2vecFG = Word2VecFeatureGenerator()
    sentiFG = SentimentFeatureGenerator()
    generators = [countFG, tfidf, svdFG, word2vecFG, sentiFG]
    
    for g in generators:
        g.process(data)
        
    for g in generators:
        g.read('train')
        
    print('done')


generate counting features
BasicCountFeatures:
       Body ID                                           Headline     Stance  \
0          712  Police find mass graves with at least '15 bodi...  unrelated   
1          158  Hundreds of Palestinians flee floods in Gaza a...      agree   
2          137  Christian Bale passes on role of Steve Jobs, a...  unrelated   
3         1034  HBO and Apple in Talks for $15/Month Apple TV ...  unrelated   
4         1923  Spider burrowed through tourist's stomach and ...   disagree   
5          154  'Nasa Confirms Earth Will Experience 6 Days of...      agree   
6          962  Accused Boston Marathon Bomber Severely Injure...  unrelated   
7         2033  Identity of ISIS terrorist known as 'Jihadi Jo...  unrelated   
8         1739  Banksy 'Arrested & Real Identity Revealed' Is ...      agree   
9          882      British Aid Worker Confirmed Murdered By ISIS  unrelated   
10        2327                                     Gateway Pundit    disc

basic counting features for training saved in train.basic.pkl
test:
                                        Headline_unigram  Body ID  \
0      [ferguson, riot, pregnant, woman, lose, eye, c...     2008   
1      [crazi, conserv, sure, gitmo, detaine, kill, j...     1550   
2      [russian, guy, say, justin, bieber, rington, s...        2   
3      [zombi, cat, buri, kitti, believ, dead, meow, ...     1793   
4      [argentina, presid, adopt, boy, end, werewolf,...       37   
5             [next, generat, appl, iphon, featur, leak]     2353   
6      [saudi, nation, airlin, may, introduc, gender,...      192   
7                  [zombi, cat, claw, way, grave, heart]     2482   
8           [isi, might, harvest, organ, iraq, tell, un]      250   
9      [woman, surgeri, get, third, breast, three, st...       85   
10     [exclus, appl, unveil, long, await, retina, ma...     1964   
11                 [50, foot, crab, crabzilla, spot, uk]     2279   
12     [found, miss, afghan, soldie

IndexError: too many indices for array

In [3]:
tfidf = TfidfFeatureGenerator()

In [4]:
tfidf.process(data)

tfidf, n_train: 49972
tfidf, n_test: 25413
xHeadlineTfidf.shape:
(75385, 769582)
headline tfidf features of training set saved in train.headline.tfidf.pkl
headline tfidf features of test set saved in test.headline.tfidf.pkl
xBodyTfidf.shape:
(75385, 769582)
body tfidf features of training set saved in train.body.tfidf.pkl
body tfidf features of test set saved in test.body.tfidf.pkl
simTfidf.shape:
(75385, 1)
tfidf sim. features of training set saved in train.sim.tfidf.pkl
tfidf sim. features of test set saved in test.sim.tfidf.pkl


1

In [3]:
svdFG = SvdFeatureGenerator()

In [4]:
svdFG.process(data)

SvdFeatureGenerator, n_train: 49972
SvdFeatureGenerator, n_test: 25413
xHeadlineTfidf.shape:
(49972, 769582)
xBodyTfidf.shape:
(49972, 769582)
simTfidf.shape:
(49972, 1)
xHeadlineTfidf.shape:
(25413, 769582)
xBodyTfidf.shape:
(25413, 769582)
simTfidf.shape:
(25413, 1)
xHeadlineTfidf.shape:
(75385, 769582)
xHeadlineSvd.shape:
(75385, 50)
headline svd features of training set saved in train.headline.svd.pkl
headline svd features of test set saved in test.headline.svd.pkl
xBodySvd.shape:
(75385, 50)
body svd features of training set saved in train.body.svd.pkl
body svd features of test set saved in test.body.svd.pkl
simSvd.shape:
(75385, 1)
svd sim. features of training set saved in train.sim.svd.pkl
svd sim. features of test set saved in test.sim.svd.pkl


1

In [5]:
word2vecFG = Word2VecFeatureGenerator()
word2vecFG.process(data)

generating word2vec features
Word2VecFeatureGenerator: n_train: 49972
Word2VecFeatureGenerator: n_test: 25413
model loaded
Headline_unigram_array:
[ ['police', 'find', 'mass', 'graves', 'with', 'at', 'least', '15', 'bodies', 'near', 'mexico', 'town', 'where', '43', 'students', 'disappeared', 'after', 'police', 'clash']
 ['hundreds', 'of', 'palestinians', 'flee', 'floods', 'in', 'gaza', 'as', 'israel', 'opens', 'dams']
 ['christian', 'bale', 'passes', 'on', 'role', 'of', 'steve', 'jobs', 'actor', 'reportedly', 'felt', 'he', 'wasn', 'right', 'for', 'part']
 ...,
 ['the', 'success', 'of', 'the', 'affordable', 'care', 'act', 'is', 'hugely', 'inconvenient', 'truth', 'for', 'its', 'opponents']
 ['the', 'success', 'of', 'the', 'affordable', 'care', 'act', 'is', 'hugely', 'inconvenient', 'truth', 'for', 'its', 'opponents']
 ['the', 'success', 'of', 'the', 'affordable', 'care', 'act', 'is', 'hugely', 'inconvenient', 'truth', 'for', 'its', 'opponents']]
(75385,)
<class 'numpy.ndarray'>
headlineV

[[  4.02333832   4.14946556   6.0451355  ...,  -4.50424957   5.85460281
   -0.20957184]
 [ 12.50268555  19.45081854  10.7670784  ..., -12.82918549  20.80779934
   -3.03554535]
 [  4.33190918   7.95248222   6.90382385 ...,  -4.62169647   8.15472221
   -1.77095032]
 ..., 
 [ 40.64879608  14.89530468  32.91280365 ...,  -7.07592154  45.26170444
  -37.51789117]
 [ 32.05303192  23.43038082  19.29140091 ..., -22.48334503  36.3652153
  -19.29603004]
 [ 20.67050171  18.20455265  15.78315735 ..., -21.48912144  37.63951874
  -15.80085373]]
(75385, 300)
bodyVec
[[ 0.03247191  0.03348987  0.04878961 ..., -0.03635329  0.04725184
  -0.00169143]
 [ 0.04067834  0.06328456  0.03503142 ..., -0.04174063  0.06769959
  -0.00987635]
 [ 0.03247841  0.0596236   0.0517613  ..., -0.03465109  0.06113989
  -0.01327767]
 ..., 
 [ 0.05757888  0.02109915  0.04662087 ..., -0.01002302  0.06411305
  -0.05314396]
 [ 0.04639235  0.03391225  0.02792164 ..., -0.03254154  0.05263364
  -0.02792834]
 [ 0.03681939  0.03242691  

1

In [6]:
sentiFG = SentimentFeatureGenerator()

In [8]:
sentiFG.process(data)

generating sentiment features
for headline
headlineSenti.shape:
(75385, 4)
headline sentiment features of training set saved in train.headline.senti.pkl
headline sentiment features of test set saved in test.headline.senti.pkl
headine senti done
for body
bodySenti.shape:
(75385, 4)
body sentiment features of training set saved in train.body.senti.pkl
body sentiment features of test set saved in test.body.senti.pkl
body senti done


1

In [3]:
countFG = CountFeatureGenerator()

In [4]:
countFG.process(data)

generate counting features
BasicCountFeatures:
       Body ID                                           Headline     Stance  \
0          712  Police find mass graves with at least '15 bodi...  unrelated   
1          158  Hundreds of Palestinians flee floods in Gaza a...      agree   
2          137  Christian Bale passes on role of Steve Jobs, a...  unrelated   
3         1034  HBO and Apple in Talks for $15/Month Apple TV ...  unrelated   
4         1923  Spider burrowed through tourist's stomach and ...   disagree   
5          154  'Nasa Confirms Earth Will Experience 6 Days of...      agree   
6          962  Accused Boston Marathon Bomber Severely Injure...  unrelated   
7         2033  Identity of ISIS terrorist known as 'Jihadi Jo...  unrelated   
8         1739  Banksy 'Arrested & Real Identity Revealed' Is ...      agree   
9          882      British Aid Worker Confirmed Murdered By ISIS  unrelated   
10        2327                                     Gateway Pundit    disc

basic counting features for training saved in train.basic.pkl
test:
                                        Headline_unigram  Body ID  \
0      [ferguson, riot, pregnant, woman, lose, eye, c...     2008   
1      [crazi, conserv, sure, gitmo, detaine, kill, j...     1550   
2      [russian, guy, say, justin, bieber, rington, s...        2   
3      [zombi, cat, buri, kitti, believ, dead, meow, ...     1793   
4      [argentina, presid, adopt, boy, end, werewolf,...       37   
5             [next, generat, appl, iphon, featur, leak]     2353   
6      [saudi, nation, airlin, may, introduc, gender,...      192   
7                  [zombi, cat, claw, way, grave, heart]     2482   
8           [isi, might, harvest, organ, iraq, tell, un]      250   
9      [woman, surgeri, get, third, breast, three, st...       85   
10     [exclus, appl, unveil, long, await, retina, ma...     1964   
11                 [50, foot, crab, crabzilla, spot, uk]     2279   
12     [found, miss, afghan, soldie